# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re
import functools

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [8]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        finish = time.perf_counter()
        print(f"Время выполнения функции {func.__name__}: {finish - start}")
        return result
    
    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Функция вызвана с параметрами:")
        print(args, kwargs)

        result = func(*args, **kwargs)
        return result

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    func._counter = 0

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Увеличиваем счетчик вызовов
        func._counter += 1
        # Выводим количество вызовов функции
        print(f"Функция была вызвана: {func._counter} раз")

        result = func(*args, **kwargs)
        return result
    
    return wrapper

## Задание 4

In [6]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    cache = {}  # Словарь для хранения кэшированных результатов

    def fmemo(*args):
        # Проверяем, есть ли результат в кэше
        if not args in cache:
            cache[args] = func(*args)
        return cache[args]

    fmemo.cache = cache
    return fmemo

## Тестирование

In [9]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция была вызвана: 1 раз
Функция вызвана с параметрами:
('whole',) {}
Время выполнения функции word_count: 3.293375900015235
Cлово whole встречается 176 раз


In [38]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [45]:
%%timeit
# измеряем время выполнения
fib(10)

12.9 μs ± 322 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [46]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [47]:
%%timeit
# измеряем время выполнения
fib(10)

128 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
